In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
from efficientnet_pytorch import EfficientNet
import numpy as np

# Define your custom dataset
class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]

        if self.transform:
            sample = self.transform(sample)

        return sample, label

# Assuming you have your data stored in numpy arrays: train_data, train_labels, test_data, test_labels



In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import os
from scipy.fft import fft,fft2, fftshift

class CustomDataset(Dataset):
    def __init__(self, img_folder, csv_file,transform=None):
        """
        Args:
            img_folder (str): Path to the folder containing images.
            csv_file (str): Path to the CSV file with image names and labels.
            transform (callable, optional): Optional transform to be applied on the image.
        """
        self.img_folder = img_folder
        self.df = pd.read_csv(csv_file)
        self.transform = transform

        # Extract image file names and labels from the CSV file
        self.img_files = self.df['Image'].values
        self.labels = self.df['any'].values

    def __len__(self):
        # Return the length of the dataset (number of samples)
        return len(self.img_files)
    def __getitem__(self, index):
    # Load the image file
        img_filename = self.img_files[index]
        img_path = os.path.join(self.img_folder, f"{img_filename}.npy")  # Construct the .npy file path

    # Load the image as a NumPy array from the .npy file
        image = np.load(img_path)
        num_angles = image.shape[0]
        num_channels = image.shape[2]
        for angle_index in range(num_angles):
                # Iterate over each channel
            for channel_index in range(num_channels):
                    # Apply Fourier transform to the current channel at the current angle
                image[angle_index, :, channel_index] = abs(fft(image[angle_index, :, channel_index]))

    # Convert the NumPy array to a PyTorch tensor

        image = torch.from_numpy(image).float()
        image=image.permute(2,0,1)

    # Apply transformations if provided
        if self.transform:
            image = self.transform(image)

    # Get the label
        label = self.labels[index]

    # Convert the label to a PyTorch tensor (if necessary)
        #label = torch.tensor(label, dtype=torch.float32)
        label = torch.tensor([label], dtype=torch.float32)

        return image, label

In [ ]:
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    # transforms.Resize((360, 362)), # Resize to match input size
    transforms.ToTensor(), # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Normalize
])

In [ ]:
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler


train_img_folder ='/mnt/DATA/sinograms_3W_train'  # Replace with your image folder path
train_csv_file = '/mnt/DATA/ee21b040/Desktop/Chandra/DLH/train_x.csv'  # Replace with your CSV file path
train_dataset = CustomDataset(train_img_folder, train_csv_file,transform=preprocess)

test_img_folder ='/mnt/DATA/EE21B040/Downloads/sinograms_3W_test' # Replace with your image folder path
test_csv_file = '/mnt/DATA/ee21b040/Desktop/Chandra/DLH/test_x.csv' # Replace with your CSV file path
test_dataset = CustomDataset(test_img_folder, test_csv_file,transform=preprocess)
test_loader = DataLoader(test_dataset, batch_size=50, shuffle=True,drop_last=True)

# Set the random seed for reproducibility (optional)
torch.manual_seed(42)

# Get the length of your train_dataset
dataset_length = len(train_dataset)

# Generate a list of indices from 0 to dataset_length
indices = list(range(dataset_length))

# Shuffle the indices
torch.random.manual_seed(42)  # Set random seed for reproducibility
# torch.random.shuffle(indices)
indices = torch.randperm(dataset_length)
indices = indices.tolist()
# Select 20,000 random indices
selected_indices = indices[:120000]

# Create a SubsetRandomSampler using the selected indices
subset_sampler = SubsetRandomSampler(selected_indices)

train_loader = DataLoader(train_dataset, batch_size=50, sampler=subset_sampler,drop_last=True)
valid_img_folder ='/mnt/DATA/sinograms_3W_train'  # Replace with your image folder path
valid_csv_file = '/mnt/DATA/ee21b040/Desktop/Chandra/DLH/valid_x.csv' # Replace with your CSV file path
valid_dataset = CustomDataset(valid_img_folder, valid_csv_file,transform=preprocess)
valid_loader = DataLoader(valid_dataset, batch_size=50, shuffle=True,drop_last=True)

In [ ]:
# Define transformations
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    # transforms.Resize((360, 362)), # Resize to match input size
    transforms.ToTensor(), # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Normalize
])

# Initialize the EfficientNet model
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=1) # 2 classes for binary classification

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


Loaded pretrained weights for efficientnet-b0


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Instantiate the model and move it to the device
model = model.to(device)
import time

Using device: cuda


In [ ]:
def save_checkpoint(epoch):
    torch.save(model.state_dict(), f'model_epoch_{epoch}.pth')


In [ ]:
import matplotlib.pyplot as plt

# Lists to store training and validation accuracies and losses
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

num_epochs = 15

# The rest of your code...

for epoch in range(num_epochs):
    model.train()

    # Start the timer for the epoch
    start_time = time.time()

    train_loss = 0.0
    train_correct = 0
    train_total = 0

    # Training loop
    for i, (images, labels) in enumerate(train_loader):
        # Move images and labels to the device (GPU or CPU)
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)

        # Reshape labels to match model output shape
        labels = labels.view(-1, 1)

        # Calculate loss
        loss = criterion(outputs, labels.float())
        train_loss += loss.item()

        # Calculate the number of correct predictions
        predictions = (outputs >= 0.5).float()  # Apply threshold to outputs
        train_correct += (predictions == labels.float()).sum().item()
        train_total += labels.size(0)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Calculate training accuracy and loss
    train_accuracy = train_correct / train_total
    train_losses.append(train_loss / len(train_loader))
    train_accuracies.append(train_accuracy)

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            # Move images and labels to the device (GPU or CPU)
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)

            labels = labels.view(-1, 1)

            # Calculate loss
            loss = criterion(outputs, labels.float())
            val_loss += loss.item()

            # Calculate the number of correct predictions
            predictions = (outputs >= 0.5).float()  # Apply threshold to outputs
            val_correct += (predictions == labels.float()).sum().item()
            val_total += labels.size(0)

    # Calculate validation accuracy and loss
    val_accuracy = val_correct / val_total
    val_losses.append(val_loss / len(valid_loader))
    val_accuracies.append(val_accuracy)

    # Print progress
    print(f'Epoch {epoch + 1}/{num_epochs}, '
          f'Train Loss: {train_losses[-1]:.4f}, '
          f'Train Accuracy: {train_accuracies[-1]:.4f}, '
          f'Validation Loss: {val_losses[-1]:.4f}, '
          f'Validation Accuracy: {val_accuracies[-1]:.4f}')

    # Calculate and print the time taken for the epoch
    end_time = time.time()
    epoch_time = end_time - start_time
    print(f'Epoch {epoch + 1} completed in {epoch_time:.2f} seconds')

# Plotting
plt.figure(figsize=(10, 5))

# Plot training and validation losses
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot training and validation accuracies
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


torch.save(model.state_dict(), 'final_model.pth')
print("Final model saved.")

In [ ]:
# Evaluation on test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = correct / total
print(f"Test Accuracy: {test_accuracy:.4f}")